# Live Bitcoin Feed Playground (BTC/USD:CXTALP)
Self-contained notebook to: (1) initialize config & connections, (2) subscribe to the live Bitcoin symbol (active on weekends), (3) stream quotes & recent candles, (4) explore frames, (5) cleanly shutdown.

**Pipeline Touchpoints**: DXLink → Redis (pub/sub + cache) + Telegraf (HTTP) → (optionally) Influx. This notebook goes directly to DXLink (bypassing FastAPI edge) for lowest latency validation.

**Stop / Rerun Guidance**: If a cell is interrupted mid-loop, re-run the subscription cell before restarting the watcher. Always run the shutdown cell before closing the notebook kernel.

In [1]:
import asyncio, logging, os, math, time, textwrap
from datetime import datetime
import pandas as pd
from IPython.display import display, clear_output, Markdown

from tastytrade.common.logging import setup_logging
from tastytrade.config import RedisConfigManager
from tastytrade.connections import Credentials, InfluxCredentials
from tastytrade.connections.subscription import RedisSubscriptionStore
from tastytrade.connections.sockets import DXLinkManager
from tastytrade.messaging.processors import (
    TelegrafHTTPEventProcessor,
    RedisEventProcessor,
)
from tastytrade.config.enumerations import Channels

# Pandas display tuning for richer exploratory output
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

logging.getLogger().handlers.clear()
setup_logging(
    level=logging.INFO,
    log_dir="../logs",
    filename_prefix="dev_tastytrade",
    console=True,
    file=True,
)
logging.getLogger("asyncio").setLevel(logging.WARNING)
logging.info("Environment ready.")

2025-08-09 17:44:53 - INFO:root:62:Logging initialized - writing to ../logs/dev_tastytrade_20250809.log
2025-08-09 17:44:53 - INFO:root:32:Environment ready.
2025-08-09 17:44:53 - INFO:root:32:Environment ready.


## Parameters

In [2]:
# Core symbol (weekend-active crypto)
SYMBOL = "BTC/USD:CXTALP"
# Candle intervals to watch (ordered smallest→largest)
CANDLE_INTERVALS = ["m", "5m", "15m", "1h", "4h", "1d"]
# Backfill start reference (adjust as desired)
BACKFILL_START = datetime.utcnow().replace(hour=0, minute=0, second=0, microsecond=0)
# Watch loop runtime (seconds); set None for indefinite (Ctrl+C to interrupt)
WATCH_SECONDS = 90
# Refresh period for live panel
REFRESH_SECS = 5
# Forward fill lookback days (for continuity)
FORWARD_FILL_LOOKBACK_DAYS = 3

SYMBOL_EVENT_PREFIX = SYMBOL  # raw symbol used by subscription helpers
logging.info("Parameters set for %s", SYMBOL)

2025-08-09 17:44:53 - INFO:root:15:Parameters set for BTC/USD:CXTALP


## Open Connections & Wire Processors

In [ ]:
config = RedisConfigManager(env_file="/workspace/.env")
config.initialize(force=True)
credentials = Credentials(config=config, env="Live")

# DXLink with Redis-backed subscription store (for replay / state)
dxlink = DXLinkManager(subscription_store=RedisSubscriptionStore())
await dxlink.open(credentials=credentials)

# Optional telemetry / distribution processors (Telegraf HTTP + Redis Pub/Sub)
for handler in dxlink.router.handler.values():
    handler.add_processor(TelegrafHTTPEventProcessor())
    handler.add_processor(RedisEventProcessor())

logging.info(
    "DXLink open; processors attached. Channels: %s", list(dxlink.router.handler.keys())
)

2025-08-09 17:44:53 - INFO:tastytrade.config.manager:174:Initialized 19 variables from .env file in Redis
2025-08-09 17:44:53 - INFO:tastytrade.connections.requests:148:Session created successfully
2025-08-09 17:44:53 - INFO:tastytrade.connections.requests:148:Session created successfully
2025-08-09 17:44:54 - INFO:tastytrade.connections.subscription:118:Redis ping response: True
2025-08-09 17:44:54 - INFO:tastytrade.connections.subscription:122:Redis version: 7.4.3
2025-08-09 17:44:54 - INFO:tastytrade.connections.subscription:123:Connected clients: 3
2025-08-09 17:44:54 - INFO:tastytrade.connections.subscription:118:Redis ping response: True
2025-08-09 17:44:54 - INFO:tastytrade.connections.subscription:122:Redis version: 7.4.3
2025-08-09 17:44:54 - INFO:tastytrade.connections.subscription:123:Connected clients: 3
2025-08-09 17:44:54 - INFO:root:14:DXLink open; processors attached. Channels: [<Channels.Control: 0>, <Channels.Quote: 7>, <Channels.Trade: 5>, <Channels.Greeks: 11>, <Cha

2025-08-09 17:44:54 - INFO:tastytrade.messaging.handlers:225:SETUP
2025-08-09 17:44:54 - INFO:tastytrade.messaging.handlers:228:AUTH_STATE:UNAUTHORIZED
2025-08-09 17:44:54 - INFO:tastytrade.messaging.handlers:228:AUTH_STATE:UNAUTHORIZED
2025-08-09 17:44:54 - INFO:tastytrade.messaging.handlers:228:AUTH_STATE:AUTHORIZED
2025-08-09 17:44:54 - INFO:tastytrade.messaging.handlers:228:AUTH_STATE:AUTHORIZED
2025-08-09 17:44:54 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:1
2025-08-09 17:44:54 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:1
2025-08-09 17:44:54 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:3
2025-08-09 17:44:54 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:3
2025-08-09 17:44:54 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:5
2025-08-09 17:44:54 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:5
2025-08-09 17:44:54 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:7
2025-08-09 17:44:54 - INFO:tastytrade.messaging.han

## Subscribe (Ticker + Candles)

In [4]:
# Ticker subscription
await dxlink.subscribe([SYMBOL])

# Candle subscriptions with bounded wait timeout
for iv in CANDLE_INTERVALS:
    coro = dxlink.subscribe_to_candles(
        symbol=SYMBOL, interval=iv, from_time=BACKFILL_START
    )
    await asyncio.wait_for(coro, timeout=15)

logging.info(
    "Subscribed to %s (ticker + %d candle intervals).", SYMBOL, len(CANDLE_INTERVALS)
)

2025-08-09 17:44:54 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP
2025-08-09 17:44:54 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{=m}
2025-08-09 17:44:54 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{=5m}
2025-08-09 17:44:54 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{=15m}
2025-08-09 17:44:54 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{=h}
2025-08-09 17:44:54 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{=4h}
2025-08-09 17:44:54 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{=d}
2025-08-09 17:44:54 - INFO:root:11:Subscribed to BTC/USD:CXTALP (ticker + 6 candle intervals).
2025-08-09 17:44:54 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{=m}
2025-08-09 17:44:54 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{

## Forward Fill Recent History (Continuity)

In [5]:
from tastytrade.utils.time_series import forward_fill

for iv in CANDLE_INTERVALS:
    event_symbol = f"{SYMBOL}{{={iv}}}"
    logging.debug("Forward-filling %s", event_symbol)
    forward_fill(symbol=event_symbol, lookback_days=FORWARD_FILL_LOOKBACK_DAYS)
logging.info("Forward fill completed.")

2025-08-09 17:44:54 - INFO:root:7:Forward fill completed.


## Plot: Recent Candles
Interactive candlestick plot utilities. Run the setup cell, then either a one-shot render or a short live updating loop. Adjust interval or lookback as needed.

In [ ]:
from tastytrade.realtime import LiveMACDHullStreamer

streamer = LiveMACDHullStreamer(symbol=SYMBOL, interval='5m', min_rows=40, poll_secs=2.0)
chart = await streamer.start(dxlink)
chart.figure

## Indicators: MACD & Hull (5m)
Live incremental MACD + Hull using production realtime module helpers (`tastytrade.realtime.notebook`). Run the setup cell below to display a live-updating chart.

In [ ]:
# The chart is already live via LiveMACDHullStreamer.
# Display figure again (it auto-updates) or access chart.figure directly.
chart.figure

KeyError: 'Candle'

## Ad-hoc Exploration

In [ ]:
# Example: access full quote frame
dxlink.router.handler[Channels.Quote].processors["feed"].pl.tail(10)

In [ ]:
# Example: access a specific candle interval frame (e.g., 5m)
iv = "5m"
frame = (
    dxlink.router.handler[Channels.Candle]
    .processors["feed"]
    .frames.get(f"{SYMBOL}{{={iv}}}")
)
frame.tail(5) if frame is not None else "No frame yet"

## Shutdown

In [ ]:
await dxlink.close()
logging.info("DXLink closed. Notebook complete.")